### 1. Imports

In [67]:
import requests
import json
import pandas as pd

### 2. Get user input 

In [68]:
# Create dict of available teams
team_dict = {'bayern': 40,
             'dortmund': 7,
             'schalke': 9}

In [69]:
home_team = input("What's the name of the home team?")
home_team = home_team.lower()

What's the name of the home team? bayern


In [70]:
away_team = input("What's the name of the away team?")
away_team = away_team.lower()

What's the name of the away team? dortmund


In [71]:
# Check if team exists
def check_team(home_team, away_team, team_dict):
    if home_team in team_dict and away_team in team_dict:
        home = team_dict[home_team]
        away = team_dict[away_team]
        return home, away
    else:
        return None, None

In [72]:
home, away = check_team(home_team, away_team, team_dict)

### 3. API call / get the data

In [73]:
# Get the match data for both teams: team1 is the HOME and team2 is the AWAY team
url = f'https://api.openligadb.de/getmatchdata/{home}/{away}'
response = requests.get(url)

### 4. Home team

In [74]:
def team_func(team, response):
    # home team is Bayern
    # check if result at the end of the match
    endergebnis = 'Endergebnis'
    # count number of matches
    match = 0
    # store if win, draw, loss in dict
    match_history = {'match':[],
                 'outcome': [],
                 'goals_team_1': [],
                 'goals_team_2': []}
    # store match_history dict in list of dicts to later transform into df

    for x in range(0, len(response.json())):
    
        try:
            if response.json()[x]['team1']['teamId'] == team:
                if endergebnis in response.json()[x]['matchResults'][0].values():
                    goals_team1 = response.json()[x]['matchResults'][0]['pointsTeam1']
                    goals_team2 = response.json()[x]['matchResults'][0]['pointsTeam2']
                    if goals_team1 > goals_team2:
                        match_history['match'].append(x)
                        match_history['goals_team_1'].append(goals_team1)
                        match_history['goals_team_2'].append(goals_team2)
                        match += 1
                    elif goals_team1 < goals_team2:
                        match_history['match'].append(x)
                        match_history['goals_team_1'].append(goals_team1)
                        match_history['goals_team_2'].append(goals_team2)
                        match += 1
                    else:
                        match_history['match'].append(x)
                        match_history['goals_team_1'].append(goals_team1)
                        match_history['goals_team_2'].append(goals_team2)
                        match += 1
                else:
                    goals_team1 = response.json()[x]['matchResults'][1]['pointsTeam1']
                    goals_team2 = response.json()[x]['matchResults'][1]['pointsTeam2']
                    if goals_team1 > goals_team2:
                        match_history['match'].append(x)
                        match_history['goals_team_1'].append(goals_team1)
                        match_history['goals_team_2'].append(goals_team2)
                        match += 1
                    elif goals_team1 < goals_team2:
                        match_history['match'].append(x)
                        match_history['goals_team_1'].append(goals_team1)
                        match_history['goals_team_2'].append(goals_team2)
                        match += 1
                    else: 
                        match_history['match'].append(x)
                        match_history['goals_team_1'].append(goals_team1)
                        match_history['goals_team_2'].append(goals_team2)
                        match += 1
        except IndexError:
            continue
    
    return match_history

In [75]:
home_match_history = team_func(home, response)
home_match_df = pd.DataFrame(home_match_history.items())

In [76]:
home_df = home_match_df.set_index(0).T

In [77]:
home_df = home_df.explode(['match', 'goals_team_1', 'goals_team_2'])
home_df = home_df[['match', 'goals_team_1', 'goals_team_2']]

In [78]:
home_df = home_df.set_index('match').reset_index()

In [79]:
home_team = [team for team, number in team_dict.items() if number == home][0]

In [80]:
home_df = home_df.rename(columns={'goals_team_1': home_team, 'goals_team_2': away_team})

### 5. Away team

In [81]:
away_match_history = team_func(away, response)
away_match_df = pd.DataFrame(away_match_history.items())

In [82]:
away_df = away_match_df.set_index(0).T

In [83]:
away_df = away_df.explode(['match', 'goals_team_1', 'goals_team_2'])
away_df = away_df[['match', 'goals_team_1', 'goals_team_2']]

In [84]:
away_df = away_df.set_index('match').reset_index()

In [85]:
away_df = away_df.rename(columns={'goals_team_1': away_team, 'goals_team_2': home_team})

### 6. Combine both df

In [86]:
def calc_new_col(row):
    if row[0] > row[1]:
        val = 1
    elif row[0] < row[1]:
        val = 2
    else:
        val = 0
    
    return val

In [87]:
def create_complete_df(home_df, away_df):
    complete_df = pd.concat([home_df, away_df])
    complete_df = complete_df.sort_values(by='match').set_index('match').reset_index()
    complete_df = complete_df.iloc[:, 1:3].astype(int)
    
    complete_df['outcome'] = complete_df.apply(calc_new_col, axis=1)
    
    return complete_df

In [88]:
complete_df = create_complete_df(home_df, away_df)

In [89]:
pd.set_option('display.max_rows', 100)
complete_df.head(71)

,bayern,dortmund,outcome
0,2,2,0
1,5,0,1
2,2,1,1
3,3,3,0
4,2,0,1
5,2,3,2
6,0,0,0
7,5,0,1
8,2,1,1
9,1,1,0
